In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import math

In [ ]:
#Connect to the IMU port
#Read Raw Data
raw = np.array([3,3])
# Store into the normal np.array list.
Rawdata = pd.DataFrame(raw)
Rawdata.to_csv('./test.csv')
# Save as a file (csv)
#np.savetxt('./test.csv', Rawdata)
#pd.write_csv(Rawdata, './test.csv')
#Load a saved csv file
Raw = pd.read_csv('./test.csv')

#관절각
#14. 토아웃각도. toe out angle 진행선/ 발의 중앙선과의 각도.
#15. 엉덩관절(고관절, Hipjoint)/ 굴곡(flexion,extension)
#16. 회선(circumduction)
#17. 무릎관절(knee joint)
#18.발목관절(ankle joint) 의 굴곡(flexion/신전 extension)

In [ ]:
#for Test purpose. Generate Psudo Data
#imudata example
#create the list with 8 vacant location

packets = [1]



for packet in packets:

    packet = ['*1', '0.1','0.2','0.3']
    #allocate correct IMUs into the array
    imus[int(imu[0][-1])-1] = imu[1:]
    
#IMU_raw = np.append(imus[0], imus[1:]) 
#Expand received pakcets into the np array
#merge as a row at the same timeline.
IMU_raw = np.vstack((IMU_raw, np.append(imus[0], imus[1:])))

IMU_raw

#This operation occurs once Save/ Analysis is called.
#Convert np.array into the pandas DataFrame and save into the CSV file.
arrays = [['imu1','imu2', 'imu3', 'imu4', 'imu5', 'imu6', 'imu7', 'imu8' ],['roll','pitch','yaw'] ]
columns = pd.MultiIndex.from_product(arrays, names=['IMUs','Coord'])

IMUdf = pd.DataFrame(IMU_raw, columns=columns)
IMUdf.to_csv('./imu_raw.csv')IMUdf.to_csv('./imu_raw.csv')



#IMUdf = pd.read_csv('./imu_raw.csv', header=[0,1], skipinitialspace=False, tupleize_cols= True)
IMUdf = pd.read_csv('./imu_raw.csv', index_col=False, names=columns, skiprows=[0,1])

#IMUdf.columns = pd.MultiIndex.from_tuples(IMUdf.columns)
#IMUdf = pd.read_csv('./imu_raw.csv', header=[0,1], skipinitialspace=False, tupleize_cols= True)
IMUdf = pd.read_csv('./imu_raw.csv', index_col=False, names=columns, skiprows=[0,1])
#IMUdf.columns = pd.MultiIndex.from_tuples(IMUdf.columns)

In [227]:
def conv2space(imuang, x = np.array([0,0,-1]), L = 1.0):
    X = x*L
    C = []
    S = []
    
    for theta in imuang:
        C.append(math.cos(theta))
        S.append(math.sin(theta))

    R_yrp = np.array([ [C[1]*C[0] - S[1]*C[2]*C[0] , -C[1]*S[0] - S[1]*C[2]*C[0] , S[1]*S[2] ],
                          [ C[1]*C[2]*C[0] -S[1]*C[0], C[1]*C[2]*C[0] - S[1]*S[0], -C[1]*S[2] ],
                         [ S[2]*S[0], S[2]*C[0], C[2] ] ])

    #Thigh / Calf/ Ankle Length

    return np.matmul(R_yrp, X)

In [ ]:
# Theta should be loaded from the measured port values
yrp = [0, 1, 2, 1] # Rotation Offset, 1, 2, 3 : Pitch Roll Yaw/Intrincsic X' Z'' Z/ => Extrincsic X, Z(1), Z(3)
#yrp = dataframe.imu[N]
IMUdf
IMUspace = IMUdf.copy(deep=True)
for imu in columns.levels[0]:
    print (IMUdf[imu] )
    IMUspace[imu] = IMUdf[imu].apply(conv2space, axis = 1)


In [238]:
IMUspace

imu1                          imu2                     imu3  \
       roll          pitch       yaw roll          pitch  yaw roll   
0  0.000000  6.949860e-310 -1.000000  0.0  6.949860e-310 -1.0  0.0   
1 -0.019834   1.976768e-01 -0.980067  0.0  6.949882e-310 -1.0  0.0   

                      imu4 ...  imu5 imu6                     imu7  \
           pitch  yaw roll ...   yaw roll          pitch  yaw roll   
0  6.949860e-310 -1.0  0.0 ...  -1.0  0.0  6.949860e-310 -1.0  0.0   
1  6.949882e-310 -1.0  0.0 ...  -1.0  0.0  6.949882e-310 -1.0  0.0   

                      imu8                      
           pitch  yaw roll          pitch  yaw  
0  6.949860e-310 -1.0  0.0  6.949860e-310 -1.0  
1  6.949882e-310 -1.0  0.0  6.949882e-310 -1.0  

[2 rows x 24 columns]

In [215]:
print (conv2space(np.array([0,0,0])))
Theta = [imu[0], imu[3], imu[1], imu[2]]

IndexError: list index out of range